# Metadados

In [159]:
from sympy import *
from itertools import *

# Einsteinpy: https://docs.einsteinpy.org/en/stable/index.html
# Sympy: https://docs.sympy.org/latest/index.html
# Intertools: https://docs.python.org/3/library/itertools.html

In [160]:
M, G, c, t, r, th, ph = symbols('M G c t r th ph')
X = [t, r, th, ph]
# Respectivamente: Massa M, constante de Newton G, velocidade da luz c, tempo t, raio r, angulo polar theta, azimuth phi 

# Solucao Schwarzschild

## Metrica

Comecaremos com a metrica

$$ g \equiv ds^2 = -\left(1 - \frac{2 G M}{c^2 r}\right) c^2 dt^2 + \frac{dr^2}{\left(1 - \frac{2 G M}{c^2 r}\right)} + r^2 d\theta^2 + r^2 \sin^2\theta d\phi^2 $$

In [161]:
# vamos definir a funcao f 
f =  1 - 2*M*G / (c**2 * r)
g = zeros(4)

In [162]:
g[0,0] = - f
g[1,1] = 1/f
g[2,2] = r**2
g[3,3] = r**2 * sin(th)**2
rank = g.rank()

In [163]:
# Inversa 
gI = g.inv()

In [164]:
g

Matrix([
[2*G*M/(c**2*r) - 1,                       0,    0,               0],
[                 0, 1/(-2*G*M/(c**2*r) + 1),    0,               0],
[                 0,                       0, r**2,               0],
[                 0,                       0,    0, r**2*sin(th)**2]])

## Simbolos de Christoffel

Esses objetos sao as componentes da conexao de Levi-Civita e sao dados explicitamente por

$$\Gamma^\rho_{\mu\nu} = \frac{1}{2} g^{\rho\sigma} \left( \partial_\mu g_{\sigma\nu} + \partial_\sigma g_{\mu\nu} - \partial_\sigma g_{\mu\nu} \right) $$

In [165]:
# k is the contravariant (upper) index, while i and j are covariant (lower) indices

def Gamma(k,i,j):
    n = 0
    GammaMatrix = 0
    while n < rank:
        GammaMatrix = GammaMatrix + gI[k,n]*(g[n,j].diff(X[i]) + g[n,i].diff(X[j]) - g[i,j].diff(X[n])) / 2
        n += 1
    return GammaMatrix

It's useful to think of these objects as 4 matrices labeled by the integer k
$$ (\Gamma^0)_{ij} \quad (\Gamma^1)_{ij} \quad (\Gamma^2)_{ij} \quad (\Gamma^3)_{ij} $$

In [166]:
ChSymb0 = zeros(rank)
ChSymb1 = zeros(rank)
ChSymb2 = zeros(rank)
ChSymb3 = zeros(rank)

In [167]:
ChSymb = [ChSymb0, ChSymb1, ChSymb2, ChSymb3]

In [168]:
for k in range(rank):
    for i in range(rank):
        for j in range(rank):
            if Gamma(k,i,j) != 0:
                ChSymb[k][i,j] = Gamma(k,i,j)

In [169]:
ChSymb[0]

Matrix([
[                        0, G*M/(r*(-2*G*M + c**2*r)), 0, 0],
[G*M/(r*(-2*G*M + c**2*r)),                         0, 0, 0],
[                        0,                         0, 0, 0],
[                        0,                         0, 0, 0]])

In [170]:
ChSymb[1]

Matrix([
[G*M*(-2*G*M + c**2*r)/(c**4*r**3),                                                           0,                       0,                                  0],
[                                0, -G*M*(-2*G*M + c**2*r)/(c**4*r**3*(-2*G*M/(c**2*r) + 1)**2),                       0,                                  0],
[                                0,                                                           0, -(-2*G*M + c**2*r)/c**2,                                  0],
[                                0,                                                           0,                       0, -(-2*G*M + c**2*r)*sin(th)**2/c**2]])

In [171]:
ChSymb[2]

Matrix([
[0,   0,   0,                0],
[0,   0, 1/r,                0],
[0, 1/r,   0,                0],
[0,   0,   0, -sin(th)*cos(th)]])

In [172]:
ChSymb[3]

Matrix([
[0,   0,               0,               0],
[0,   0,               0,             1/r],
[0,   0,               0, cos(th)/sin(th)],
[0, 1/r, cos(th)/sin(th),               0]])

## Riemann Tensor

We now have all the Chrystoffel symbols (it's a courtesy of python!), so that we can calculate the Riemann tensor

$$ R^l_{kij} = \partial_i \Gamma^l_{kj} - \partial_j \Gamma^l_{ki} + \Gamma^l_{ip}\Gamma^p_{kj} -  \Gamma^l_{jp}\Gamma^p_{ki} $$

In [173]:
# l is the contravariant index, k,i,j are covariant indices
# I'm gonna use the objects ChSymb because I don't want to compute these symbols at each run
def Contraction(l,k,i,j):
    n = 0 
    cont = 0 
    while n < rank:
        cont = cont + ChSymb[l][i,n] * ChSymb[n][k,j] - ChSymb[l][j,n] * ChSymb[n][k,i]
        n += 1
    return cont

In [174]:
def Riem(l,k,i,j):
    n = 0
    Riemann = 0
    while n < rank:
        Riemann = Riemann + ChSymb[l][k,j].diff(X[i]) - ChSymb[l][k,i].diff(X[j]) + Contraction(l,k,i,j)
        n += 1
    R = Riemann.simplify()
    return R

Now we can finally calculate the components of the Riemann tensor. It is useful to consider the components as a matrix of matrices $(R^l_k)_{ij}$

In [175]:
RieTen00 = zeros(rank); RieTen01 = zeros(rank); RieTen02 = zeros(rank); RieTen03 = zeros(rank)
RieTen10 = zeros(rank); RieTen11 = zeros(rank); RieTen12 = zeros(rank); RieTen13 = zeros(rank)
RieTen20 = zeros(rank); RieTen21 = zeros(rank); RieTen22 = zeros(rank); RieTen23 = zeros(rank)
RieTen30 = zeros(rank); RieTen31 = zeros(rank); RieTen32 = zeros(rank); RieTen33 = zeros(rank)

In [176]:
Rie = [[RieTen00, RieTen01, RieTen02, RieTen03], 
       [RieTen10, RieTen11, RieTen12, RieTen13],
       [RieTen20, RieTen21, RieTen22, RieTen23],
       [RieTen30, RieTen31, RieTen32, RieTen33]]

In [177]:
for l in range(rank):
    for k in range(rank):
        for i in range(rank):
            for j in range(rank):
                if Riem(l,k,i,j) != 0:
                    Rie[l][k][i,j] = Riem(l,k,i,j)

In [178]:
# Run this cell to see the matrices explicitly
for l in range(rank):
    for k in range(rank):
        display(Rie[l][k])

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

Matrix([
[                            0, 8*G*M/(r**2*(-2*G*M + c**2*r)), 0, 0],
[8*G*M/(r**2*(2*G*M - c**2*r)),                              0, 0, 0],
[                            0,                              0, 0, 0],
[                            0,                              0, 0, 0]])

Matrix([
[             0, 0, -4*G*M/(c**2*r), 0],
[             0, 0,               0, 0],
[4*G*M/(c**2*r), 0,               0, 0],
[             0, 0,               0, 0]])

Matrix([
[                        0, 0, 0, -4*G*M*sin(th)**2/(c**2*r)],
[                        0, 0, 0,                          0],
[                        0, 0, 0,                          0],
[4*G*M*sin(th)**2/(c**2*r), 0, 0,                          0]])

Matrix([
[                                 0, 8*G*M*(-2*G*M + c**2*r)/(c**4*r**4), 0, 0],
[8*G*M*(2*G*M - c**2*r)/(c**4*r**4),                                   0, 0, 0],
[                                 0,                                   0, 0, 0],
[                                 0,                                   0, 0, 0]])

...

Matrix([
[0, 0,                          0,                         0],
[0, 0,                          0,                         0],
[0, 0,                          0, 8*G*M*sin(th)**2/(c**2*r)],
[0, 0, -8*G*M*sin(th)**2/(c**2*r),                         0]])

Matrix([
[                                  0, 0, 0, 4*G*M*(2*G*M - c**2*r)/(c**4*r**4)],
[                                  0, 0, 0,                                  0],
[                                  0, 0, 0,                                  0],
[4*G*M*(-2*G*M + c**2*r)/(c**4*r**4), 0, 0,                                  0]])

Matrix([
[0,                             0, 0,                              0],
[0,                             0, 0, 4*G*M/(r**2*(-2*G*M + c**2*r))],
[0,                             0, 0,                              0],
[0, 4*G*M/(r**2*(2*G*M - c**2*r)), 0,                              0]])

Matrix([
[0, 0,              0,               0],
[0, 0,              0,               0],
[0, 0,              0, -8*G*M/(c**2*r)],
[0, 0, 8*G*M/(c**2*r),               0]])

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

## Ricci Tensor

Now we can finally calculate the Ricci tensor. Spoiler alert: Since the Schwarzschild is a vacuum solution, all components of the Ricci tensor must vanish. 
$$ R_{ij} = R^k_{ikj}$$

In [179]:
def Ricci(i,j):
    ricc = 0
    n = 0
    for k in range(rank):
        ricc = ricc + Rie[k][i][k,j]
        r = ricc.simplify()
    return r

In [180]:
Ric = [Ricci(i,j) for i in range(rank) for j in range(rank)]

In [181]:
Ric

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

## Invariante de Kretschmann

In order to verify the nature of the divergence, we need to calculate a metric invariant, in particular, the Kretschmann invariant which is defined as

$$ K = R^{m_0 n_0 p_0 q_0} R_{m_0 n_0 p_0 q_0} = g_{m_0 m_1} g^{n_0 n_1} g^{p_0 p_1} g^{q_0 q_1} R^{m_0}_{n_1 p_1 q_1} R^{m_1}_{n_0 p_0 q_0} $$

In [194]:
indices = list(product([0,1,2,3], repeat=2))
def Kr():
    K = 0
    for m in indices:
        for n in indices:
            for p in indices: 
                for q in indices:
                    K += g[m[0],m[1]] * gI[n[0],n[1]] * gI[p[0],p[1]] * gI[q[0],q[1]] * Rie[m[0]][n[1]][p[1], q[1]] * Rie[m[1]][n[0]][p[0],q[0]]
    K = K.simplify() / 16 # Esse 16 corrige as redundancias que o 'for loop' gera nessas contas
    return K

In [195]:
Kr()

48*G**2*M**2/(c**4*r**6)